# 第3回講義 宿題

## 課題

今Lessonで学んだことを元に，MNISTのファッション版 (Fashion MNIST，クラス数10) を多層パーセプトロンによって分類してみましょう．

Fashion MNISTの詳細については以下のリンクを参考にしてください．

Fashion MNIST: https://github.com/zalandoresearch/fashion-mnist

### 目標値

Accuracy 85%

### ルール

- 訓練データはx_train， t_train，テストデータはx_testで与えられます．
- 予測ラベルは one_hot表現ではなく0~9のクラスラベル で表してください．
- **下のセルで指定されているx_train，t_train以外の学習データは使わないでください．**
- Pytorchを第3回の演習を参考に，NumPyを用いて実装をしてください．

### 提出方法

- 2つのファイルを提出していただきます．
  - テストデータ (x_test) に対する予測ラベルをcsvファイル (ファイル名: submission_pred.csv) で提出してください．
  - それに対応するpythonのコードをsubmission_code.pyとして提出してください (%%writefileコマンドなどを利用してください)．

### 評価方法

- 予測ラベルのt_testに対する精度 (Accuracy) で評価します．
- 提出後即時採点を行い，Leader Boardが更新されます．
- 締切後の点数を最終的な評価とします．

In [2]:
# ドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### データの読み込み

- この部分は修正しないでください

In [2]:
import os
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import inspect


#学習データ
x_train = np.load('drive/MyDrive/Colab Notebooks/DLBasics2023_colab/Lecture03/data/x_train.npy')
t_train = np.load('drive/MyDrive/Colab Notebooks/DLBasics2023_colab/Lecture03/data/y_train.npy')
    
#テストデータ
x_test = np.load('drive/MyDrive/Colab Notebooks/DLBasics2023_colab/Lecture03/data/x_test.npy')

# データの前処理（正規化， one-hot encoding)
x_train, x_test = x_train / 255., x_test / 255.
x_train, x_test = x_train.reshape(x_train.shape[0], -1), x_test.reshape(x_test.shape[0], -1)
t_train = np.eye(N=10)[t_train.astype("int32").flatten()]

FileNotFoundError: ignored

### 多層パーセプトロンの実装

In [3]:
import os
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import inspect


#学習データ
x_train = np.load('drive/MyDrive/Colab Notebooks/DeepLearning/Lecture03/data/x_train.npy')
t_train = np.load('drive/MyDrive/Colab Notebooks/DeepLearning/Lecture03/data/y_train.npy')
    
#テストデータ
x_test = np.load('drive/MyDrive/Colab Notebooks/DeepLearning/Lecture03/data/x_test.npy')

# データの前処理（正規化， one-hot encoding)
x_train, x_test = x_train / 255., x_test / 255.
x_train, x_test = x_train.reshape(x_train.shape[0], -1), x_test.reshape(x_test.shape[0], -1)
t_train = np.eye(N=10)[t_train.astype("int32").flatten()]

In [4]:
# データの分割
x_train, x_val, t_train, t_val =\
    train_test_split(x_train, t_train, test_size=10000)

In [5]:
def np_log(x):
    return np.log(np.clip(x, 1e-10, 1e+10))


def create_batch(data, batch_size):
    """
    :param data: np.ndarray，入力データ
    :param batch_size: int，バッチサイズ
    """
    num_batches, mod = divmod(data.shape[0], batch_size)
    batched_data = np.split(data[: batch_size * num_batches], num_batches)
    if mod:
        batched_data.append(data[batch_size * num_batches:])

    return batched_data

In [6]:
# シード値を変えることで何が起きるかも確かめてみてください．
rng = np.random.RandomState(1234)
random_state = 42

# 発展: 今回の講義で扱っていない活性化関数について調べ，実装してみましょう
def relu(x):
  return np.maximum(x,0)


def deriv_relu(x):
  return (x > 0).astype(x.dtype)


def softmax(x):
  x -= x.max(axis=1, keepdims=True)
  x_exp = np.exp(x)
  return x_exp / x_exp.sum(axis=1, keepdims=True)


def deriv_softmax(x):
  return softmax(x) * (1-softmax(x))


def crossentropy_loss(t, t_hat):
  return (- t * np_log(t_hat)).sum(axis=1).mean()

class Dense:
  def __init__(self, in_dim, out_dim, function, deriv_function):

    self.W = np.random.uniform(low=-0.08, high=0.08,
                              size=(in_dim, out_dim)).astype("float64")
    self.b = np.zeros(out_dim).astype("float64")
    self.function = function
    self.deriv_function = deriv_function

    self.x = None
    self.u = None

    self.dW = None
    self.db = None

    self.params_idxs = np.cumsum([self.W.size, self.b.size])

  def __call__(self, x):
    self.x = x
    self.u = np.matmul(self.x, self.W) + self.b
    h = self.function(self.u)
    return h

  def b_prop(self, delta, W):
    self.delta = self.deriv_function(self.u) * np.matmul(delta, W.T)
    return self.delta

  def compute_grad(self):
    batch_size = self.delta.shape[0]
    self.dW = np.matmul(self.x.T, self.delta) / batch_size
    self.db = np.matmul(np.ones(batch_size), self.delta) / batch_size

  def get_params(self):
    return np.concatenate([self.W.ravel(), self.b], axis=0)

  def set_params(self, params):
    _W, _b = np.split(params, self.params_idxs)[:-1]
    self.W = _W.reshape(self.W.shape)
    self.b = _b

  def get_grads(self):
    return np.concatenate([self.dW.ravel(), self.db], axis=0)


class Model:
    def __init__(self, hidden_dims, activation_functions, deriv_functions):
        """
        :param hiden_dims: List[int]，各層のノード数を格納したリスト．
        :params activation_functions: List, 各層で用いる活性化関数を格納したリスト．
        :params derive_functions: List，各層で用いる活性化関数の導関数を格納したリスト．
        """
        # 各層をリストに格納していく
        self.layers = []
        self.adam = Adam()

        for i in range(len(hidden_dims)-2):  # 出力層以外は同じ構造
            self.layers.append(Dense(hidden_dims[i], hidden_dims[i+1],
                                     activation_functions[i], deriv_functions[i]))
        self.layers.append(Dense(hidden_dims[-2], hidden_dims[-1],
                                 activation_functions[-1], deriv_functions[-1]))  # 出力層を追加

    def __call__(self, x):
        return self.forward(x)

    def forward(self, x):
        """順伝播処理を行うメソッド"""
        for layer in self.layers:
            x = layer(x)
        return x

    def backward(self, delta):
        """誤差逆伝播，勾配計算を行うメソッド"""
        batch_size = delta.shape[0]

        for i, layer in enumerate(self.layers[::-1]):
            if i == 0:  # 出力層の場合
                layer.delta = delta  # y - t
                layer.compute_grad()
            else:  # 出力層以外の場合
                delta = layer.b_prop(delta, W)  # 逆伝播
                layer.compute_grad()  # 勾配の計算

            W = layer.W

    def adam_opt(self, lr= 0.01):
      for layer in self.layers:        
          params = {'W': layer.W, 'b': layer.b}
          grads = {'W': layer.dW, 'b': layer.db}
          self.adam.update(params, grads)



    def update(self, eps=0.1):
        """パラメータの更新を行うメソッド"""
        for layer in self.layers:
            layer.W -= eps * layer.dW
            layer.b -= eps * layer.db




In [7]:
class Adam:
    def __init__(self, lr=0.01, beta1=0.9, beta2=0.09):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.iter = 0
        self.m = None
        self.v = None

    def update(self, params, grads):
        if self.m is None:
            """
            更新初回にインスタンス変数 m, vを生成
            """
            self.m, self.v = {}, {}
            for key, val in params.items():
                self.m[key] = np.zeros_like(val)
                self.v[key] = np.zeros_like(val)
        
        self.iter += 1
        lr_t  = self.lr * np.sqrt(1.0 - self.beta2**self.iter) / (1.0 - self.beta1**self.iter)         
        
        for key in params.keys():
            self.m[key] = self.beta1 * self.m[key] + (1 - self.beta1) * grads[key]
            self.v[key] = self.beta2 * self.v[key] + (1 - self.beta2) * (grads[key]**2)
            m_unbias = self.m[key] / (1 - self.beta1**self.iter)
            v_unbias = self.v[key] / (1 - self.beta2**self.iter)
            
            params[key] -= lr_t * self.m[key] / (np.sqrt(self.v[key]) + 1e-7)


### モデルの学習

In [9]:
from tqdm import tqdm

def train_model(mlp, x_train, t_train, x_val, t_val, n_epochs=10, lr=0.1):
    for epoch in tqdm(range(n_epochs)):
        losses_train = []
        losses_valid = []
        train_num = 0
        train_true_num = 0
        valid_num = 0
        valid_true_num = 0


        x_train, t_train = shuffle(x_train, t_train)
        x_train_batches, t_train_batches = create_batch(x_train, batch_size), create_batch(t_train, batch_size)

        x_val, t_val = shuffle(x_val, t_val)
        x_val_batches, t_val_batches = create_batch(x_val, batch_size), create_batch(t_val, batch_size)

        # モデルの訓練
        for x, t in zip(x_train_batches, t_train_batches):
            # 順伝播
            t_hat = mlp(x)

            # 損失の計算
            loss = crossentropy_loss(t,t_hat)

            # パラメータの更新
            delta = t_hat - t
            mlp.backward(delta)
            losses_train.append(loss.tolist())
            mlp.update(eps=lr)
            #mlp.adam_opt(lr)


            # 精度を計算
            acc = accuracy_score(t.argmax(axis=1), t_hat.argmax(axis=1), normalize=False)
            train_num += x.shape[0]
            train_true_num += acc

        # モデルの評価
        for x, t in zip(x_val_batches, t_val_batches):
            # 順伝播
            y = mlp(x)

            # 損失の計算
            loss = crossentropy_loss(t,y)
            losses_valid.append(loss.tolist())

            acc = accuracy_score(t.argmax(axis=1), y.argmax(axis=1), normalize=False)
            valid_num += x.shape[0]
            valid_true_num += acc.sum().item()

        if epoch % 10 == 9 or epoch == 0:
          print('EPOCH: {}, Train [Loss: {:.3f}, Accuracy: {:.3f}], Valid [Loss: {:.3f}, Accuracy: {:.3f}]'.format(
              epoch + 1,
              np.mean(losses_train),
              train_true_num/train_num,
              np.mean(losses_valid),
              valid_true_num/valid_num
          ))



batch_size = 128
n_epochs = 2000
lr = 0.001

mlp = Model(hidden_dims=[784, 516, 128, 10],
              activation_functions=[relu, relu, relu, softmax],
              deriv_functions=[deriv_relu, deriv_relu, deriv_relu, deriv_softmax])
print('BatchSize: {}, N_epochs: {}, eps: {}'.format(
      batch_size,
      n_epochs,
      lr
))
train_model(mlp, x_train, t_train, x_val, t_val, n_epochs, lr)

BatchSize: 128, N_epochs: 2000, eps: 0.001


  0%|          | 1/2000 [00:13<7:27:19, 13.43s/it]

EPOCH: 1, Train [Loss: 2.222, Accuracy: 0.244], Valid [Loss: 2.142, Accuracy: 0.343]


  0%|          | 10/2000 [01:15<3:57:48,  7.17s/it]

EPOCH: 10, Train [Loss: 0.949, Accuracy: 0.688], Valid [Loss: 0.929, Accuracy: 0.686]


...

EPOCH: 1980, Train [Loss: 0.083, Accuracy: 0.979], Valid [Loss: 0.346, Accuracy: 0.895]


100%|█████████▉| 1990/2000 [3:45:47<01:05,  6.58s/it]

EPOCH: 1990, Train [Loss: 0.083, Accuracy: 0.979], Valid [Loss: 0.339, Accuracy: 0.896]


100%|██████████| 2000/2000 [3:46:55<00:00,  6.81s/it]

EPOCH: 2000, Train [Loss: 0.081, Accuracy: 0.980], Valid [Loss: 0.345, Accuracy: 0.895]


In [ ]:
t_pred = []
for x in x_test:
    # 順伝播
    x = x[np.newaxis, :]
    y = mlp(x)

    # モデルの出力を予測値のスカラーに変換
    pred = y.argmax(1).tolist()

    t_pred.extend(pred)

submission = pd.Series(t_pred, name='label')
submission.to_csv('drive/MyDrive/Colab Notebooks/DeepLearning/Lecture03/submission_pred.csv', header=True, index_label='id')